In [8]:
from pathlib import Path

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [9]:
data_dir = Path.cwd().parent / "data"
df = pd.read_csv(data_dir / "factors.csv", parse_dates=["date"])

In [10]:
factor_cols = ["ret_geo", "vol_36m", "value", "investment", "profitability"]

factor = df.replace([np.inf, -np.inf], np.nan)

In [11]:
factor.isna().sum()

PERMNO             0
date               0
tic                0
conm               0
market_cap         0
n_shares           0
n_months           0
ret_geo            0
vol_36m            0
value            770
profitability    770
investment         0
dtype: int64

### EDA

In [ ]:
def winsorize_cross_section(df, cols, lower=0.005, upper=0.995):
    df = df.copy()
    for col in cols:
        # Apply separately for each year (cross-sectional clean)
        df[col] = df.groupby(by="date")[col].transform(
            lambda x: x.clip(lower=x.quantile(lower), upper=x.quantile(upper))
        )
    return df


factor_winsorized = winsorize_cross_section(factor, factor_cols).sort_values(
    ["date", "PERMNO"]
)

### Factor returns

In [ ]:
factor_cols = {
    "V": "value",  # value
    "W": "ret_geo",  # momentum signal
    "C": "investment",  # investment
    "R": "profitability",  # profitability
    "L": "vol_36m",  # low volatility
}

ret_col = "ret_geo"  # return used for portfolio performance
w_col = "market_cap"  # value weights


def value_weighted_return(g):
    """Value weighted return of ret_col using w_col within a group."""
    if g.empty:
        return np.nan
    return np.average(g[ret_col], weights=g[w_col])


def assign_terciles(x):
    """Assign 0 (bottom), 1 (middle), 2 (top) tercile based on ranks within a date."""
    ranks = x.rank(method="first")
    t1 = len(x) / 3.0
    t2 = 2.0 * t1
    out = pd.Series(index=x.index, dtype="float")
    out[ranks <= t1] = 0
    out[(ranks > t1) & (ranks <= t2)] = 1
    out[ranks > t2] = 2
    return out


# Build factor portfolio return series for each factor
factor_ret_list = []

for short_name, score_col in factor_cols.items():
    df = factor_winsorized[["date", ret_col, w_col, score_col]].copy()

    # Assign terciles per date on the factor signal
    df["tercile"] = df.groupby("date")[score_col].transform(assign_terciles)

    # Top and bottom terciles
    top = df[df["tercile"] == 2]
    bottom = df[df["tercile"] == 0]

    # Value weighted returns by date
    top_ret = (
        top.groupby("date").apply(value_weighted_return).rename(short_name + "_top")
    )

    bottom_ret = (
        bottom.groupby("date").apply(value_weighted_return).rename(short_name + "_bot")
    )

    # Factor return = top minus bottom
    fr = pd.concat([top_ret, bottom_ret], axis=1)
    fr[short_name] = fr[short_name + "_top"] - fr[short_name + "_bot"]

    # Keep only the factor return series
    factor_ret_list.append(fr[[short_name]])

# 2. Combine all factor return series into one dataframe
factor_returns = pd.concat(factor_ret_list, axis=1)
factor_returns = factor_returns.sort_index()

C:\Users\chris\AppData\Local\Temp\ipykernel_27672\984464648.py:61: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top.groupby("date").apply(value_weighted_return).rename(short_name + "_top")
C:\Users\chris\AppData\Local\Temp\ipykernel_27672\984464648.py:65: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bottom.groupby("date").apply(value_weighted_return).rename(short_name + "_bot")


ValueError: cannot reindex on an axis with duplicate labels

In [42]:
factor_returns.head(10)

,V,W,C,R,L
date,,,,,
1963-01-31,0.018702,0.027356,0.011811,0.025640,0.025640
1963-02-28,-0.002044,0.009215,0.001875,-0.002044,-0.005891
1963-03-29,-0.011790,0.038703,-0.011831,-0.006966,0.009045
1963-04-30,-0.012700,0.035412,0.007204,-0.007406,0.000926
1963-05-31,-0.002727,0.013192,0.000530,-0.017610,0.008107
1963-06-28,0.002672,0.023143,0.000673,0.000555,0.005356
1963-07-31,0.006583,0.012314,0.002446,0.002368,0.005195
1963-08-30,-0.009509,0.023200,-0.009509,-0.009509,0.023200
1963-09-30,-0.014551,0.020918,0.003309,-0.016603,0.021165


In [47]:
X = factor_returns[["V", "W", "C", "R", "L"]].copy()

vif_table = pd.DataFrame(
    {
        "factor": X.columns,
        "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])],
    }
)

print(vif_table)

  factor       VIF
0      V  1.603379
1      W  1.326063
2      C  1.266702
3      R  1.187321
4      L  1.294022


In [ ]:
factors_cols = ["V", "C", "W", "R", "L"]
df = factor_returns[factors_cols]

results = pd.DataFrame(index=factors_cols, columns=factors_cols)

for dep in factors_cols:
    X = df[factors_cols].drop(columns=[dep])  # independent vars
    X = sm.add_constant(X)
    y = df[dep]

    model = sm.OLS(y, X).fit()
    R2 = model.rsquared
    VIF = 1 / (1 - R2)

    # store VIF in the dependent variable's COLUMN
    for ind in X.columns:
        if ind != "const":
            results.loc[ind, dep] = round(VIF, 2)  # matches paper formatting

# diagonal should be blank or dashes
for f in factors_cols:
    results.loc[f, f] = "-"

print(results)


      V     C     W     R     L
V     -  1.18  1.14  1.19  1.19
C  1.46     -  1.14  1.19  1.19
W  1.46  1.18     -  1.19  1.19
R  1.46  1.18  1.14     -  1.19
L  1.46  1.18  1.14  1.19     -
